# KNN-1: train with partial curves 

My idea is to used one of the partial cycle files to train the KNN model. For that, I need to get the distances between each of the partial curves with all the full curves. So for each partial curve, choose the closer full curve.
The result is an array with all the distances for each partial cycle. The number of cycle is the label of each distance.
With that i train the model.

Then, the user will give us a curve. We need to get first the distance of this curve to all our full curves and get the closer one.
That distance is what i feed the model to make a prediction of the cycle number.

We can used one file of partial curves as train set and then other as test set.

**I tried to do this, but the code for getting the distance for each partial curve takes FOREVER!. So i only used 2 partial curves to train. Of course this is not okey, but i just wanted to see if this made any sense**

In [2]:
import pandas as pd
import numpy as np
import import_data
import sort_data
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline

/Users/rossana13/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#Load the data. We need full curves and one partial curve (that represents the user's curve)

PL03c,PL03d = sort_data.charge_discharge('converted_PL03.mat')
full_curvesc,full_curvesd = sort_data.charge_discharge('converted_PL11.mat')


In [6]:
a = PL03c[444]['voltage'].values.flatten()
b = full_curvesc[2][['voltage']].values.flatten()

In [7]:
print(len(a),len(b))

802 1209


In [36]:
#Make a shorter partial curve dictionary to uses as train set (because if not the code takes too long)
PL03c_short = {}
for idx in range(1,3):
        if idx not in PL03c_short.keys():
            PL03c_short[idx] = PL03c[idx]
PL03c_short           

{1:               time       datetime  step  cycle  current_amp   voltage  \
 12       60.000548  735943.605567   2.0    1.0     0.747940  4.228518   
 13       65.016632  735943.605625   3.0    1.0     0.079300  4.200349   
 14       70.032166  735943.605683   3.0    1.0     0.072414  4.200349   
 15       75.032202  735943.605741   3.0    1.0     0.066978  4.200349   
 16       80.048026  735943.605799   3.0    1.0     0.062629  4.200188   
 17       85.063402  735943.605856   3.0    1.0     0.059368  4.200349   
 18       90.063472  735943.605914   3.0    1.0     0.056650  4.200349   
 19       95.078710  735943.605972   3.0    1.0     0.054113  4.200511   
 20      100.079078  735943.606030   3.0    1.0     0.051938  4.200349   
 21      105.079125  735943.606088   3.0    1.0     0.049945  4.200349   
 22      110.094682  735943.606146   3.0    1.0     0.048496  4.200511   
 23      115.094783  735943.606204   3.0    1.0     0.046684  4.200349   
 24      120.110254  735943.606262 

In [32]:
PL03c_short[1]['voltage'].values.flatten()

array([4.22851849, 4.20034933, 4.20034933, 4.20034933, 4.20018768,
       4.20034933, 4.20034933, 4.20051146, 4.20034933, 4.20034933,
       4.20051146, 4.20034933, 4.20051146, 4.20034933, 4.20034933,
       4.20051146, 4.20034933, 4.20051146, 4.20034933, 4.20051146,
       4.20051146, 4.20034933, 4.20018768, 4.20051146, 4.20034933,
       4.20034933, 4.20051146, 4.20034933, 4.20034933, 4.20034933,
       4.20034933, 4.20034933, 4.20018768, 4.20018768, 4.20018768,
       4.20034933, 4.20034933, 4.20034933, 4.20034933, 4.20034933,
       4.20051146, 4.20034933, 4.20034933, 4.20034933, 4.20034933,
       4.20034933, 4.20018768, 4.20051146, 4.20034933, 4.20034933,
       4.20034933, 4.20034933, 4.20051146, 4.20051146, 4.20034933,
       4.20034933, 4.20051146, 4.20034933, 4.20034933, 4.20051146,
       4.20034933, 4.20034933, 4.20034933, 4.20034933, 4.20034933,
       4.20034933, 4.20034933, 4.20034933, 4.20034933, 4.20051146,
       4.20034933, 4.20034933, 4.20051146, 4.20034933, 4.20051

In [9]:
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def curve_distance(a,b):
#Inputs are two nparray    
    distance,path = fastdtw(a, b, dist=euclidean)
    return distance

In [48]:
def distance_cycle_to_full(cycle,dic1,dic2):
#Inputs are 2 dictionaries and the number of cycle that is being tested
    if cycle not in dic1.keys():
        pass
    else:
        
        a = dic1[cycle][['voltage']].values.flatten()
        Y = []
    
        for idx in range(1,len(dic2.keys()) + 1):
            if idx in dic2.keys():
                if idx > 100:
                    b = dic2[idx][['voltage']].values.flatten() 
                    d = curve_distance(a,b)
            
                    A = [idx,d]
                    Y.append(A)
                else:
                    pass
            else:
                pass
    df = pd.DataFrame(Y, columns = ['Cycle','Distance'])
    df = df.sort_values(by = ['Distance'])
    
    Distance = df['Distance'][df.index[0]]
    return Distance

In [49]:
def partial_to_full(dic1,dic2):
    C = []
    D = []
    for i in range(1,len(dic1.keys()) + 1):
        if i in dic1.keys():
            Distance = distance_cycle_to_full(i,dic1,dic2)
            C.append(i)
            D.append(Distance)
        else:
            pass
    #For the model, we need the lists as NumPy arrays
    D = np.array(D)
    C = np.array(C)
    return D,C

In [33]:
distance_to_full(1,PL03c_short,full_curvesc)

155.18576312065125

In [37]:
distance_to_full(2,PL03c_short,full_curvesc)

81.13978791236877

In [38]:
partial_to_full(PL03c_short,full_curvesc)

([155.18576312065125, 81.13978791236877], [1, 2])

In [46]:
D = np.array([155.18576312065125, 81.13978791236877])
C = np.array([1, 2])

In [47]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
neigh.fit(D.reshape(-1, 1), C) #Train the classifier with the distance and the corresponding cycles

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [52]:
#For predicting first we need the distance between the testing curve to the full curves
def distance_to_full(curve,dic):
 
    #Convert the data in the data frame to a NumPy array
    a = curve.values.flatten() 
    Y = []
    
    #Calculate the distances between the testing curve and each of the full curve cycle
    for i in range(1,len(dic.keys()) + 1):
        if i in dic.keys():
            if i > 100:
                b = dic[i][['voltage']].values.flatten() 
                d = curve_distance(a,b)
            
                A = [i,d]
                Y.append(A)
            else:
                pass
        else:
            pass
    #Creat a Pandas dataframe with the distances for each cycle. Sort the data frame by distance
    df = pd.DataFrame(Y, columns = ['Cycle','Distance'])
    df = df.sort_values(by = ['Distance'])
    return df

In [54]:
df = distance_to_full(PL03c[444]['voltage'],full_curvesc)

In [56]:
Dist = df['Distance'][df.index[0]]
Dist

80.87822318077087

In [57]:
neigh.predict(Dist)

array([2])